In [210]:
"""
red-black tree

properties:
1. Every node is either red or black;
2. Root node is black;
3. Every Nil leaf is black;
4. A red node has two black children nodes;
5. Every path from root to leaf has same number of black nodes.

It has a 1 to 1 correspondence to a 2-3 tree
"""


# base node class
class Node:
    def __init__(self,value):
        self._value = value
        self._left = None
        self._right = None
        self._parent = None
        
    # properties
    @property
    def value(self):
        return self._value
    
    @property
    def left(self):
        return self._left
    
    @property
    def right(self):
        return self._right
        
    @property
    def parent(self):
        return self._parent
        
    @property
    def color(self):
        return self._color
    
    # setters
    @left.setter
    def left(self,l):
        self._left = l

    @right.setter
    def right(self,r):
        self._right = r

    @parent.setter
    def parent(self,p):
        self._parent = p

    @color.setter
    def color(self,color):
        self._color = color
    
    def _red(self, v):
        return "\x1b[31m" + v +"\x1b[0m"

    def _black(self,v):
        return v
    
    def _white(self,v):
        return "\x1b[37m" + v + "\x1b[0m"
    
    # print node value and color
    def __str__(self,level=0, offset=0, prefix = ""):
        if self._value == None:
            v = "None"
        else:
            v = repr(self._value)
        if self._color == 'RED':
            text = self._red(v)
        elif self._color == 'BLACK': 
            text = self._black(v) 
        return "     "*level + self._white(prefix) + text

    
    
    
# nil class
class Nil(Node):
    def __init__(self):
        Node.__init__(self, None)
        self._color = 'BLACK'

        
# tree node class       
class RBNode(Node):
    def __init__(self,value):
        Node.__init__(self, value)
        self._left = Nil()
        self._right = Nil()
        self._parent = Nil()
        self._color = None

        
        
        
# tree class
class RBTree(): 
    # all leaf nodes's children is merged to one Nil
    # root's parent is also that nil 
    def __init__(self):
        self._root = Nil()
        self._nil = Nil()
        self._root.parent = self._nil
    
    
    # walk all nodes and apply method
    def _inorder_walk(self, node, level, offset, prefix, method):
        # walk if node is RBNode
        # or Nil Node
        if isinstance(node, RBNode):
        # or isinstance(node, Nil)
            self._inorder_walk(node.left, level+1, offset-1, "/-- ", method)
            method(node, level, offset, prefix)
            self._inorder_walk(node.right, level+1, offset+1, "\\-- ", method)
            
            
    # print all nodes 
    def display(self):
        print_method = lambda n,l,o,p: print(n.__str__(l,o,p))
        return self._inorder_walk(self._root, 0, 0, "    ", print_method)

            
            
    # left rotation operation
    # example:
    #    X            Y
    #   / \          / \
    #  α   Y   =>   X   γ 
    #     / \      / \
    #    β   γ    α   β
    # It changes pointer structure only.
    def _left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left :
            y.left.parent = x
        y.parent = x.parent
        if x.parent == self._nil:
            self._root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y
        
        
    # right rotation operation symmetric to left operation
    #      Y            X     
    #     / \          / \     
    #    X   γ   =>   α   Y  
    #   / \              / \
    #  α   β            β   γ   
    def _right_rotate(self, y):
        x = y.left
        y.left = x.right
        if x.right != self._nil:
            x.right.parent = y
        x.parent = y.parent
        if y.parent == self._nil:
            self._root = x
        elif y == y.parent.right:
            y.parent.right = x
        else:
            y.parent.left = x
        x.right = y
        y.parent = x
    
    
    #
    def _transplant(self, u, v):
        # when u is root
        if u.parent == self._nil:
            self._root = v
        # when u is its parent's left children
        elif u.parent.left == u:
            u.parent.left = v
        # u is its parent's right children
        else:
            u.parent.right = v
        v.parent = u.parent

        
    # min node in its subtree
    def _min(self, node):
        while node.left != self._nil:
            node = node.left
        return node
    
    
    # max node in its subtree
    def _max(self, node):
        while node.right != self._nil:
            node = node.right
        return node        
    
    
    # find successor of node
    # either in its right subtree
    # or in its first parent on the right
    def _successor(self, node):
        if node.right != self._nil:
            return self._min(node.right)
        else:
            p = node.parent
            while p != self._nil and  p.right == node:
                node = p
                p = p.parent
            return p
    
    
    # find predecessor
    def _predecessor(self, node):
        if node.left != self._nil:
            return self._max(node.left)
        else:
            p = node.parent
            while p != self._nil and p.left == node:
                node = p
                p = p.parent
            return p
        

    # find node of particular value
    def search(self, value):
        node = self._root
        while node != self._nil and node.value != value:
            if node.value < value:
                node = node.right
            elif node.value > value:
                node = node.left
        return node
 

    # insert a new node into tree
    def insert(self, node):
        # almose-same-as-binary-search-tree start
        value = node.value
        n = self._root
        parent = self._nil
        while not isinstance(n ,Nil):
            parent = n
            if n.value > value:
                n = n.left
            else:
                n = n.right
        node.parent = parent
        if parent == self._nil:
            self._root = node
        else:
            if parent.value > value:
                parent.left = node
            else:
                parent.right = node
        # additional
        node.left = self._nil
        node.right = self._nil
        node.color = 'RED'
        # almose-same-as-binary-search-tree end
        
        # fix up property 4 violation
        z = node
        while z.parent.color == 'RED':
            # parent is left child, red uncle
            # or parent is right child, red uncle
            if z.parent == z.parent.parent.left:
                y = z.parent.parent.right
                if y.color == 'RED':
                    # case 1
                    z.parent.parent.color = 'RED'                
                    z.parent.parent.left.color = 'BLACK'
                    y.color = 'BLACK'
                    z = z.parent.parent
                    # continue while
                else:
                    # case 2
                    if z == z.parent.right: 
                        z = z.parent
                        self._left_rotate(z)
                    # case 3
                    z.parent.color = 'BLACK'
                    z.parent.parent.color = 'RED'
                    self._right_rotate(z.parent.parent)
                    # stop while

            # symmetric as the above "if"
            elif z.parent == z.parent.parent.right:
                y = z.parent.parent.left
                if y.color == 'RED':
                    # case 4
                    z.parent.parent.color = 'RED'                
                    z.parent.parent.right.color = 'BLACK'
                    y.color = 'BLACK'
                    z = z.parent.parent
                    # continue while
                else:
                    # case 5
                    if z == z.parent.left:
                        z = z.parent
                        self._right_rotate(z)
                    # case 6
                    z.parent.color = 'BLACK'
                    z.parent.parent.color = 'RED'
                    self._left_rotate(z.parent.parent)
                    # stop while
                        
        # fix up property 2 violation               
        self._root.color = 'BLACK'
      
    
    def delete(self, z):
        y = z
        c = y.color
        # case 1: has only right child
        # do transplant right away
        if z.left == self._nil:
            x = z.right
            self._transplant(z, z.right)
        # case 2: has only left child
        # do transplant right away
        elif z.right == self._nil:
            x = z.left
            self._transplant(z, z.left)
        # case 3 and 4: has two children
        else:
            y = self._min(z.right)
            c = y.color
            x = y.right
            if y.parent == z:
                # case 3
                # successor is node's right child
                x.parent = y
            else:
                # case 4
                # if successor is not node's right child, but in the subtree
                # then move it to node's place
                print('case 4')
                self._transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self._transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color

        # fixup
        if c == 'BLACK':
            while x != self._nil and x.color == 'BLACK':
                if x == x.parent.left:
                    w = x.parent.right
                    # case 5, sibling is RED
                    if w.color == 'RED':
                        w.color == 'BLACK'
                        x.parent.color == 'RED'
                        self._left_rotate(x.parent)
                        w = x.parent.right
                    # case 6, sibling is Black
                    # and has two Black children
                    if w.left.color == 'BLACK' and w.right.color == 'BLACK':
                        w.color = 'RED'
                        x = x.parent
                    else:
                        # case 7, sibling is Black, left child is RED, right is Black
                        if w.left.color == 'RED' and w.right.color == 'BLACK':
                            w.left.color = 'BLACK'
                            w.color = 'RED'
                            self._right_rotate(w)
                            w = x.parent.right
                        # case 8
                        w.color = x.parent.color
                        x.parent.color = 'BLACK'
                        w.right.color = 'BLACK'
                        self._left_rotate(x.parent)
                        break
                # symmetric
                elif x == x.parent.right:
                    w = x.parent.left
                    if w.color == 'RED':
                        w.color == 'BLACK'
                        x.parent.color == 'RED'
                        self._right_rotate(x.parent)
                        w = x.parent.left
                    if w.right.color == 'BLACK' and w.left.color == 'BLACK':
                        w.color = 'RED'
                        x = x.parent
                    else:
                        if w.right.color == 'RED' and w.left.color == 'BLACK':
                            w.right.color = 'BLACK'
                            w.color = 'RED'
                            self._left_rotate(w)
                            w = x.parent.left
                        w.color = x.parent.color
                        x.parent.color = 'BLACK'
                        w.right.color = 'BLACK'
                        self._right_rotate(x.parent)
                        break
            x.color = 'BLACK'  

In [211]:
# test
print("INIT red-black tree")
rbt = RBTree()

print("\n\n 1. INSERT")
for i in [2,3,5,4,0,6,1,9,7,8]:
    print("\n INSERT {0}\n".format(i))
    rbt.insert(RBNode(i))  
    rbt.display()

print("\n\n 2. DELETE")
for j in [4,1,9,5]:
    n = rbt.search(j)
    print("\n DELETE {0}\n".format(j))
    rbt.delete(n)
    rbt.display()

INIT red-black tree
1. INSERT 



 INSERT 2

    2

 INSERT 3

    2
     \-- 3

 INSERT 5

     /-- 2
    3
     \-- 5

 INSERT 4

     /-- 2
    3
          /-- 4
     \-- 5

 INSERT 0

          /-- 0
     /-- 2
    3
          /-- 4
     \-- 5

 INSERT 6

          /-- 0
     /-- 2
    3
          /-- 4
     \-- 5
          \-- 6

 INSERT 1

          /-- 0
     /-- 1
          \-- 2
    3
          /-- 4
     \-- 5
          \-- 6

 INSERT 9

          /-- 0
     /-- 1
          \-- 2
    3
          /-- 4
     \-- 5
          \-- 6
               \-- 9

 INSERT 7

          /-- 0
     /-- 1
          \-- 2
    3
          /-- 4
     \-- 5
               /-- 6
          \-- 7
               \-- 9

 INSERT 8

               /-- 0
          /-- 1
               \-- 2
     /-- 3
          \-- 4
    5
          /-- 6
     \-- 7
               /-- 8
          \-- 9
2. DELETE 



 DELETE 4

               /-- 0
          /-- 1
               \-- 2
     /-- 3
    5
          /-- 6
     \